In [ ]:
# Manage imports
from pathlib import Path
import GpxAudioLabeler as gwl
import LabelerUtilities as lu

In [ ]:
# Assign data home directory, collection filename, sampling filepath, and clip home directory
data_home = str(Path("~").expanduser() / "Data" / "AISonobuoy")
collection_filename = "collection-gpx.json"
sampling_filepath = str(Path("GpxAudioLabeler").parent / "data" / "sampling-gpx.json")
clip_home = Path("~").expanduser() / "Datasets" / "AISonobuoy"

In [ ]:
# Load file describing the collection
collection_path = Path(data_home) / collection_filename
collection = lu.load_json_file(collection_path)

In [ ]:
# Identify the interval during which any source emitted
src_min_start_t = min([h["start_t"] for h in collection["sources"]]) * 1000  # [ms]
src_max_stop_t = max([h["stop_t"] for h in collection["sources"]]) * 1000  # [ms]

# Identify the interval during which all hydrophone collected
hyd_max_start_t = (
    max([h["start_t"] for h in collection["hydrophones"]]) * 1000
)  # [ms]
hyd_min_stop_t = (
    min([h["stop_t"] for h in collection["hydrophones"]]) * 1000
)  # [ms]

In [ ]:
# Load file describing the sampling cases
# NOTE: Edit this file to change parameters
sampling = lu.load_json_file(sampling_filepath)

In [ ]:
# Consider the first source
source = collection["sources"][0]
gpx_path = Path(data_home) / source["name"]
gpx, vld_t, vld_lambda, vld_varphi, vld_h = gwl.parse_source_gpx_file(gpx_path, source)

In [ ]:
# Consider each hydrophone
hydrophone = collection["hydrophones"][0]
wav_path = Path(data_home) / hydrophone["name"]
audio = lu.get_audio_file(wav_path)

In [ ]:
# Export audio with no source present, if it exists
if src_max_stop_t < hyd_min_stop_t:
    lu.export_audio_clip(
        audio,
        src_max_stop_t,
        hyd_min_stop_t,
        Path(clip_home) / f"{Path(hydrophone['name'].lower()).stem}-no-source.wav",
    )

In [ ]:
# Compute and plot source metrics for the current hydrophone
(
    distance,
    heading,
    heading_dot,
    speed,
    r_s_h,
    v_s_h,
) = lu.compute_source_metrics(source, vld_t, vld_lambda, vld_varphi, vld_h, hydrophone)

lu.plot_source_metrics(
    source, hydrophone, heading, heading_dot, distance, speed, r_s_h
)

In [ ]:
# Consider the first sampling case
case = sampling[0]
clip_home = Path(clip_home) / case["output_dir"]
if not clip_home.exists():
    clip_home.mkdir(parents=True)
method = case["method"]

In [ ]:
if method["type"] == "clusters":
    (
        distance_clusters,
        heading_clusters,
        heading_dot_clusters,
        speed_clusters,
    ) = lu.cluster_source_metrics(
        distance,
        method["distance_n_clusters"],
        heading,
        method["heading_n_clusters"],
        heading_dot,
        method["heading_dot_n_clusters"],
        speed,
        method["speed_n_clusters"],
    )
    gwl.slice_source_audio_by_cluster(
        hydrophone,
        audio,
        hyd_max_start_t,
        hyd_min_stop_t,
        vld_t,
        r_s_h,
        distance_clusters,
        heading_clusters,
        heading_dot_clusters,
        speed_clusters,
        case["delta_t_max"],
        case["n_clips_max"],
        clip_home,
        do_plot=False,
    )
elif method["type"] == "conditionals":
    gwl.slice_source_audio_by_condition(
        hydrophone,
        audio,
        hyd_max_start_t,
        hyd_min_stop_t,
        vld_t,
        r_s_h,
        distance,
        method["distance_limits"],
        heading,
        method["heading_limits"],
        heading_dot,
        method["heading_dot_limits"],
        speed,
        method["speed_limits"],
        case["delta_t_max"],
        case["n_clips_max"],
        clip_home,
        do_plot=False,
    )